<a href="https://colab.research.google.com/github/pvrancx/pytorch_utils/blob/master/pytorch_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [0]:
!pip3 install torch torchvision

Mount google drive to store results

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
! mkdir -p "gdrive/My Drive/cifar100"

In [0]:
% cd gdrive/My Drive/cifar100

Import utilities repository from GitHub

In [0]:
! git clone https://github.com/pvrancx/pytorch_utils.git

In [0]:
% cd pytorch_utils

In [0]:
! git pull

Setup experiment

In [0]:
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models.resnet import Bottleneck, ResNet

In [0]:
from torchutils.experiment import Experiment, DataLoaders, Config
from torchutils.train import fit
from torchutils.callbacks import ModelSaverCallback, LoggerCallback, CallbackHandler

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [0]:
def get_dataloaders(batch_size=256):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    training_data = datasets.CIFAR100(
        root='./data', train=True, download=True, transform=transform_train
        )
    train_loader = DataLoader(
        training_data, 
        shuffle=True, 
        batch_size=batch_size
        )

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    test_data = datasets.CIFAR100(
        root='./data', train=False, download=True, transform=transform_test
        )
    test_loader = DataLoader(
        test_data, 
        shuffle=False, 
        batch_size=batch_size
        )
    return DataLoaders(train=train_loader, test=test_loader)

In [0]:
def get_experiment():
  resnet50 = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=100)
  optimizer = torch.optim.Adam(resnet50.parameters())
  return Experiment(
      model=resnet50,
      optimizer=optimizer,
      lr_scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer),
      loss_fn=nn.CrossEntropyLoss(),
      config=Config(device=device, max_epochs=200)
  )


In [ ]:
! mkdir -p "/content/gdrive/My Drive/cifar100/checkpoints"

In [0]:
def get_callbacks():
    return CallbackHandler([
        LoggerCallback(),
        ModelSaverCallback("/content/gdrive/My Drive/cifar100/checkpoints", frequency=10)]
    )

Fit model

In [0]:
fit(exp=get_experiment(), data=get_dataloaders(), callbacks=get_callbacks())
